In [1]:
import os
import logging
import torch
from huggingface_hub import login
from dotenv import load_dotenv
from diffusers import DiffusionPipeline


/Users/ulysses/anaconda3/envs/llms/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [2]:
%pip install diffusers transformers torch accelerate peft

Note: you may need to restart the kernel to use updated packages.


In [3]:
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s')
logger = logging.getLogger(__name__) # Create a logger for this module

# --- Authentication
load_dotenv()
hf_token = os.getenv("HF_TOKEN")
if not hf_token:
    logger.warning("HF_TOKEN environment variable not found. Some operations might fail.")
else:
    try:
        login(hf_token, add_to_git_credential=True)
        logger.info("Successfully logged into Hugging Face Hub.")
    except Exception as e:
        logger.error(f"Failed to log into Hugging Face Hub: {e}")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
2025-05-23 19:25:54,743 - WARNING - huggingface_hub._login - Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
2025-05-23 19:25:54,744 - INFO - __main__ - Successfully logged into Hugging Face Hub.


In [4]:
pipe = DiffusionPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-dev"
    )
pipe = pipe.to("mps")
pipe.load_lora_weights("multimodalart/isometric-skeumorphic-3d-bnb")

#prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"
#prompt = "Home with fusion architecture of vietnamese and mexican styles, warm colors, detailed, 4k"
#prompt = "Ragnarok online female inquisitor, fighting stance, rampage blast, white hair, wearing a Duneyrr Helm, detailed, 4k"

# Isometric Prompt when using weight ~ Comment it on when not using the model
#prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, RBNBICN, icon, white background, isometric perspective"
#prompt = "Home with fusion architecture of vietnamese and mexican styles, warm colors, detailed, 4k, RBNBICN, icon, white background, isometric perspective"
#prompt = "Ragnarok online female inquisitor, fighting stance, rampage blast, detailed, 4k, RBNBICN, icon, white background, isometric perspective"
prompt = "Ragnarok online Prontera City, detailed, 4k, RBNBICN, icon, white background, isometric perspective"

# Add some debug prints to check device placement
print(f"Pipeline device: {pipe.device}")

def clear_memory():
    # Clear MPS cache
    if torch.backends.mps.is_available():
        torch.mps.empty_cache()
    
    # Force garbage collection
    import gc
    gc.collect()
    
    # Log memory status
    logger.info("Memory cleared successfully")

image = pipe(
    prompt,
    #height=1024,
    #width=1024,
    # guidance_scale=3.5,
    # num_inference_steps=50,
    # max_sequence_length=512,
    # generator=torch.Generator("mps").manual_seed(0)  # Changed from "cpu" to "mps"
).images[0]
image.save("prontera-city-isometric.png")
logger.info("Image created successfully")
clear_memory() 

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


Pipeline device: mps:0


  0%|          | 0/28 [00:00<?, ?it/s]

2025-05-23 20:51:27,656 - INFO - __main__ - Image created successfully
2025-05-23 20:51:28,193 - INFO - __main__ - Memory cleared successfully
